# Comparison: Reinforced learning vs Dynamic programming

Dynamic programming can be used to solve a life cycle model, as shown by Määttänen (2013). Here we compare a rather simple grid based method to solve _unemployment-v0_ environment, and compare the results againt those obtained by Reinforced Learning.

Tarkastellussa elinkaarimallissa _unemployment-v0_ on huomioitu vain kolme tilaa: työssä, työtön ja vanhuuseläkkeellä. Jokainen henkilö tekee vuosittain päätöksen työhönosallistumisesta ja alimman vanhuuseläkeiän täyttämisen jälkeen valinnan työn, työttömyyden ja vanhuuseläkkeen välillä. 

Mallissa palkat ovat stokastisia.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle, DynProgLifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.5 valittaa paljon)
import warnings
warnings.filterwarnings('ignore')

size1=10_000_000
batch1=32
batch2=1_00
pop=10_000
train_dyn=False
sim_dyn=False
gamma=0.92
debug=False
plotdebug=False
rlmodel='trpo'

file='best/dynamic_prog_V_minimal_gamma092.h5'
savedfile='results/dynamic_prog_results_minimal_092'
rl_res='results/trpo_res'
rl_save='best/trpo'

# Reinforced Learning

Reinforced learning on toinen tapa laskea tuloksia elinkaarimallista. Tässä tarkastelemme kahta eri versiota: Softmax- ja Deterministic. Softmax-mallissa (deterministic=False) käytetään softmax-todennäköisyyksiä seuraavan toimen valinnassa. Deterministisessä valitaan todennäköisin toimi. 

Deterministinen on näistä kahdesta lähestymistavasta lähempänä taloustieteen utiliteetin maksimointia. Osoittautuukin, että tämä vastaa paremmin hilalaskelmia.

## Stokastinen aktionvalinta softmaxilla

Toiminnan tilassa voi valita joko stokastisesti todennäköisyyksien mukaan softmax-funktiolla tai deterministisesti.

In [ ]:
cc6=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0,gamma=gamma,
              plotdebug=plotdebug,exploration=True,exploration_ratio=0.2)
cc6.explain()
#cc6.run_results(debug=False,steps1=size1,pop=pop,deterministic=False,
#                train=False,predict=True,batch1=batch1,save='saved/malli_perusmini99_nondet',
#                plot=True,cont=True,start_from='saved/malli_perusmini99_nondet',
#                results='results/mini_simut_res_softmax',rlmodel='small_acktr')



In [ ]:
cc6.run_results(debug=debug,steps1=size1,pop=pop,deterministic=True,
                train=True,predict=False,batch1=batch1,save=rl_save,
                plot=True,cont=False,start_from=rl_save,plotdebug=plotdebug,log_interval=100,
                results=rl_res,rlmodel=rlmodel,learning_rate=0.0003, learning_schedule='linear')

In [ ]:
cc6.run_results(debug=debug,steps1=50_000,pop=pop,deterministic=True,
                train=False,predict=True,batch1=batch2,save=rl_save_sto,
                plot=True,cont=True,start_from=rl_save,plotdebug=plotdebug,
                results=rl_res,rlmodel=rlmodel)

# Tuloksia

In [ ]:
cc1=Lifecycle(env='unemployment-v0',minimal=True,mortality=False,perustulo=False,
              randomness=True,plotdebug=False,timestep=1.0)
cc1.render(load=rlsto)

In [ ]:
#cc3.compare_act(66,cc1,rlmodel='small_acktr',load='saved/malli_perusmini99_nondet',deterministic=True)

In [ ]:
cc3b=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0,gamma=gamma)
cc3b.load_sim(savedfile)
cc3b.load_V(file)
cc3b.compare_ages(cc1,rlmodel=rlmodel,load=rl_save_sto,
                 deterministic=True,time_in_state=1)

# Dynaaminen ohjelmointi

Ajetaan elämänkaarimallia sekä dynaamisella ohjelmoinnilla. Verrataan tuloksia, jotta näemme, miten hyvin RL toimii. Ajoajat eivät ole kovin vertailukelpoisia.

Dynaaminen ohjelmointi-koodi toimii ainoastaan minimaalisen mallin kanssa.

In [ ]:
cc3=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0,gamma=gamma)

In [ ]:
if train_dyn:
    cc3.train(save=file,debug=False)

cc3.load_V(file)
cc3.explain()
if sim_dyn:
    cc3.simulate(pop=pop,save=savedfile)
else:
    cc3.load_sim(savedfile)

#cc3.plot_V(2)
#cc3.render()

In [ ]:
cc3.render()

In [ ]:
cc3.print_q(68,time_in_state=0)

In [ ]:
cc3.print_actV(65)

In [ ]:
cc3.print_V(70)

In [ ]:
cc3.plot_actV_diff(30)

In [ ]:
cc3.plot_Hila(30,emp=0)
cc3.plot_Hila(30,emp=1)

In [ ]:
cc3.plot_Hila(30,emp=1)

In [ ]:
cc3.plot_Hila(30,diff=True)

In [ ]:
age=65
cc3b.plot_actHila(age,emp=0,diff=True,time_in_state=1,emp2=1)
cc3b.plot_actHila(age,emp=0,diff=True,time_in_state=1,emp2=2)
cc3b.plot_actHila(age,emp=1,diff=True,time_in_state=1,emp2=1)
cc3b.plot_actHila(age,emp=1,diff=True,time_in_state=1,emp2=2)

In [ ]:
cc3b.plot_actHila(67,emp=1,diff=False,time_in_state=1,act=0)
cc3b.plot_actHila(67,emp=1,diff=False,time_in_state=1,act=1)
cc3b.plot_actHila(67,emp=1,diff=False,time_in_state=1,act=2)

## Deterministinen

Deterministinen ennustaminen käyttää samaa sovitetta kuin stok

In [ ]:
cc7=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0,gamma=gamma)
cc7.explain()
cc7.run_results(debug=False,steps1=size1,pop=pop,deterministic=True,
                train=False,predict=True,batch1=batch1,save=rl_save_sto,
                plot=True,cont=True,start_from=rl_save_sto,
                results=rldet,rlmodel=rlmodel,twostage=False)
cc7.render()


# Päätöksenteon vertailu

Lasketaan vertailukohta RL-menetelmällä ACKTR ja katsotaan paljonko tulokset eroavat.

Verrataan ensin hilamallin tuloksia softmax-versioon.

In [ ]:
#cc6=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc7=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc6.render(load='results/mini_simut_res_softmax')
#cc7.render(load='results/mini_simut_res')
#cc7.compare_with(cc6)
#cc5.compare_with(cc3)
cc6.compare_with(cc3)

Sitten verrataan "determinististä" sovitetta.

In [ ]:
cc1.compare_with(cc3)

Tehdään sama RL-menetelmällä Deep Q-learning (dqn).

Deterministinen aktion valinta.

In [ ]:
#cc6=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc6.train(steps=1_000_000,cont=False,rlmodel='acktr',save='miniperus')
#cc6.simulate(pop=10_000,deterministic=True,load='miniperus',rlmodel='acktr',save='results/acktr_tulokset_miniperus_det')
#cc7=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc6.train(steps=1_000_000,cont=False,rlmodel='acktr',save='miniperus')
#cc7.simulate(pop=2_000,deterministic=True,load='miniperus',rlmodel='dqn')

In [ ]:
cc0=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc0.load_sim(rlsto)
cc1=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc1.load_sim(rldet)
#cc1.render()
cc2=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc2.load_sim(savedfile)
#cc2.render()

#cc0.compare_with(cc1,label1='softmax',label2='deterministic')
cc0.compare_with(cc2,label1='softmax',label2='DP')
#cc1.compare_with(cc2,label1='deterministic',label2='DP')

In [ ]:
cc1.compare_with(cc2)

In [ ]:
cc2.render()
cc1.render()
cc0.render()

# Päätöksenteon vertailu

Vertaillaan eri mallien tuloksia ja niiden eroja.

In [ ]:
cc6.compare_with(cc3b)

In [ ]:
cc7.compare_with(cc3b)

In [ ]:
cc6.compare_with(cc7)

In [ ]:
cc=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
ika=55
cc.plot_actV(ika,emp=1,time_in_state=0)
cc.RL_simulate_V(ika,rlmodel='acktr',emp=1,time_in_state=0,load='miniperus')
cc.RL_simulate_V(ika,rlmodel='acktr',emp=0,time_in_state=1,load='miniperus')
cc.RL_simulate_V(ika,rlmodel='acktr',emp=2,time_in_state=0,load='miniperus')

# Jakaumat

Reinforced Learningiin sisältyy aina epävarmuutta. Sen määrää tuloksissa voi tarkastella esimerkiksi ajamalla saman mallin monta kertaa ja vertaamalla tuloksia.

In [ ]:
size1=10_000_000
size2=0
batch1=32
batch2=0
pop_size=1_000
dire='results/v0_qlearn/'
deterministic=True
n=20

In [ ]:
cc2=Lifecycle(env='unemployment-v0',minimal=True,mortality=False,perustulo=False,
              randomness=True,plotdebug=False,version=0,timestep=1.0)



In [ ]:
cc2.run_distrib(n=n,startn=0,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=drl,plot=False,cont=True,start_from=rl_save_sto,results=dire+'distrib_base',
                rlmodel=rlmodel,twostage=False)



In [ ]:
cc2.comp_distribs(load=dire+'distrib_base',n=n,startn=0,stats_results=dire+'distrib_stats')

In [ ]:
cc2=Lifecycle(env='unemployment-v0',minimal=True,mortality=False,perustulo=False,
              randomness=True,plotdebug=False)

cc2.render_distrib(stats_results=dire+'distrib_stats',figname='peruskuva_')